In [1]:
import logging
import pandas as pd
import os
import json

In [2]:
from retrieve_data_by_query import RetrieveDataByQuery
# t = RetrieveDataByQuery(sessionid="72311449442%3AH6DGmjoHPSEFBt%3A9%3AAYfhjmVCEbQqQuRrNtjfPFjsWGdPMJAlrnjGYEAV2A")
# t = RetrieveDataByQuery(sessionid="72308001558%3AqHVUuGTZxfgCYE%3A22%3AAYeeiENJVL-8yqGoOrMZKS0xnNjyrE-_1KliGUuz9w")
# t = RetrieveDataByQuery(sessionid="71968904949%3AnasVvoUgWlaAh9%3A24%3AAYd4CsZNXogQYrfEcx5z1MFKBGij2-m8LTppQKPwfA")
t = RetrieveDataByQuery(sessionid="67725660272%3ANXdSMubsMWGOE7%3A24%3AAYcbjhQJnV3WpjTo--cwlMk6LprCuS0_46p-6NYD3g")
# t.retrieve_follower_by_userid_after("42058280528", str(25 * 1))
# 729950933

In [3]:
query = "Artificial Intelligence"

In [4]:
def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


progress_tracker = {}

def save_progress(userid: str, batch_number: int, results: list):
    """Salva il batch_number corrente e i risultati ottenuti per un determinato userid."""
    progress_tracker[userid] = {"batch_number": batch_number, "results": results}
    with open("progress.json", "w") as f:
        json.dump(progress_tracker, f)

def load_progress(userid: str):
    """Carica il batch_number e i risultati salvati per un determinato userid."""
    try:
        with open("progress.json", "r") as f:
            data = json.load(f)
            return data.get(userid, {"batch_number": 1, "results": []})
    except (FileNotFoundError, json.JSONDecodeError):
        return {"batch_number": 1, "results": []}
def remove_progress(userid: str):
    """Rimuove il progresso salvato per un determinato userid."""
    try:
        with open("progress.json", "r") as f:
            data = json.load(f)
        
        if userid in data:
            del data[userid]
            with open("progress.json", "w") as f:
                json.dump(data, f)
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
def get_all_follower(userid: str):
    progress = load_progress(userid)
    batch_number = progress["batch_number"]
    results = progress["results"]
    allpk = [res["pk"] for res in results]
    try:
        if batch_number == 1:
            response = t.retrieve_follower_by_userid(userid)
            data = response.get("data", {}).get("user", {}).get("followers", {})
            d_out = get_follower_data(data.get("edges", []))
            for res in d_out:
                allpk.append(res["pk"])
            results.extend(d_out)
            has_next_page = data.get("page_info", {}).get("has_next_page", False)
        else:
            has_next_page = True

        while has_next_page:
            try:
                print("Pagina:", batch_number)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                d_out = get_follower_data(data.get("edges", []))
                for res in d_out:
                    if res["pk"] in allpk:
                        remove_progress(userid)
                        return results
                    allpk.append(res["pk"])
                results.extend(d_out)
                
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
                
                # Salva il progresso dopo ogni batch
                save_progress(userid, batch_number, results)
                
            except Exception as e:
                save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")
    
    except Exception as e:
        save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore generale
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    remove_progress(userid)
    return results




def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

In [5]:


# results = []
# response = t.retrieve_thread_by_query(query)
# data = response.get("data", {}).get("searchResults", {})
# results.extend(get_post_data(data.get("edges", [])))
# has_next_page = data.get("page_info", {}).get("has_next_page", False)
# after = data.get("page_info", {}).get("end_cursor")
# i = 1000

# while has_next_page and i > 0:
#     response = t.retrieve_thread_by_query_after(query, after)
#     data = response.get("data", {}).get("searchResults", {})
#     results.extend(get_post_data(data.get("edges", [])))
#     has_next_page = data.get("page_info", {}).get("has_next_page", False)
#     after = data.get("page_info", {}).get("end_cursor")
#     i -= 1

# # Creazione del DataFrame
# df = pd.DataFrame(results)
# df.shape

In [6]:
# df.drop_duplicates(inplace=True)
# df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')
# df.to_csv("data.csv", index=False)

In [7]:
df = pd.read_csv("data_parte1.csv")
df['user_pk'] = df['user_pk'].astype(str)
unique_user_pk = df['user_pk'].unique()
len(unique_user_pk)

349

In [8]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "followers_data.csv"
processed_users = load_processed_users(csv_file)

In [9]:
def fetch_followers(pk):
    print(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"]
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None



follower_list = []
batch_size = 1


print("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        print(f"Skipping already processed user_pk: {user_pk}")
        continue
    
    processed_users.update([user_pk])
    results = fetch_followers(user_pk)
    if results is None:
        break
    follower_list.extend(results)
    
    if follower_list:
        print(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []

if follower_list:
    print(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)
    save_to_csv(follower_df, csv_file)

print("Follower data collection completed")

Starting follower data collection
Skipping already processed user_pk: 187264532
Skipping already processed user_pk: 39643192594
Skipping already processed user_pk: 3312160968
Skipping already processed user_pk: 623659295
Skipping already processed user_pk: 6018955246
Skipping already processed user_pk: 71440324623
Skipping already processed user_pk: 1175659953
Skipping already processed user_pk: 37065216050
Skipping already processed user_pk: 295418393
Skipping already processed user_pk: 273811002
Skipping already processed user_pk: 45697224377
Skipping already processed user_pk: 241752404
Skipping already processed user_pk: 66897403288
Skipping already processed user_pk: 429303617
Skipping already processed user_pk: 5854904084
Skipping already processed user_pk: 1624593034
Skipping already processed user_pk: 56347813790
Skipping already processed user_pk: 46153597924
Skipping already processed user_pk: 55388117027
Skipping already processed user_pk: 5924884086
Skipping already process

In [10]:
df1 = pd.read_csv("followers_data.csv")
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
df_count

(348, 3)


,follower_pk,follower_username,follower_count
user_pk,,,
70,48,48,48
5411,884,884,884
374099,50,50,50
381039,134,134,134
387960,248,248,248
...,...,...,...
70995803294,365,365,365
71003131428,110,110,110
71440324623,254,254,254
